In [ ]:
# Make the necessary imports
import hist
from hist import Hist
import ctypes
import math
import numpy as np

# 4: Special histogram axes

## 4.1: Transforms

Based on [boost-histogram](https://github.com/scikit-hep/boost-histogram)'s Transform, hist provides a powerful transform system on Regular axes that allows you to provide a functional form for the conversion between a regular spacing and the actual bin edges. The following transforms are built in:

- `hist.axis.transform.sqrt`: A square root transform;
- `hist.axis.transform.log`: A logarithmic transform;
- `hist.axis.transform.Pow(power)`: Raise to a specified power;
- `hist.axis.transform.Function`: Specify arbitrary conversion functions.

In [ ]:
axis0 = hist.axis.Regular(10, 1, 4, name="A", transform=hist.axis.transform.sqrt)
axis1 = hist.axis.Regular(10, 1, 4, name="B", transform=hist.axis.transform.log)
axis2 = hist.axis.Regular(10, 1, 4, name="C", transform=hist.axis.transform.Pow(2))

ftype = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double)
axis3 = hist.axis.Regular(
    10,
    1,
    4,
    name="D",
    transform=hist.axis.transform.Function(ftype(math.log), ftype(math.exp)),
)
axis4 = hist.axis.Regular(
    10,
    1,
    4,
    name="E",
    transform=hist.axis.transform.Function(ftype(np.log), ftype(np.exp)),
)

In [ ]:
h = Hist(axis0, axis1, axis2, axis3, axis4)

Hist also provides transform shortcuts for histograms. hist's keeps the features of boost-histogram, and you can pass transform as a keyword argument when creating an axis. hist also allows you to use the `.new` shortcut directly when creating a Histogram for common transforms:

In [ ]:
h0 = Hist.new.Sqrt(10, 1, 4).Sqrt(10, 4, 9).Double()
h1 = Hist.new.Log(10, 1, 4).Log(10, 4, 9).Double()
h2 = Hist.new.Pow(10, 1, 4, power=3).Pow(10, 1, 4, power=5).Double()

h3 = (
    Hist.new.Func(10, 1, 4, forward=ftype(math.log), inverse=ftype(math.exp))
    .Func(10, 1, 4, forward=ftype(np.log), inverse=ftype(np.exp))
    .Double()
)

## 4.3: Axis Types

Based on boost-histogram’s Axis, hist support six types of axis, `Regular`, `Boolean`, `Variable`, `Integer`, `IntCategory` and `StrCategory` with additional names and labels.

Names are pretty useful for some histogramming shortcuts, thus greatly facilitate HEP’s studies. Note that the name is the identifier for an axis in a histogram and must be unique.

In [ ]:
axis0 = hist.axis.Regular(10, -5, 5, overflow=False, underflow=False, name="A")
axis1 = hist.axis.Boolean(name="B")
axis2 = hist.axis.Variable(range(10), name="C")
axis3 = hist.axis.Integer(-5, 5, overflow=False, underflow=False, name="D")
axis4 = hist.axis.IntCategory(range(10), name="E")
axis5 = hist.axis.StrCategory(["T", "F"], name="F")

Histogram is consisted with various axes, there are two ways to create a histogram, currently. You can either fill a histogram object with axes or add axes to a histogram object. You cannot add axes to an existing histogram. _Note that to distinguish these two method, the second way has different axis type names (abbr.)._

In [ ]:
# fill the axes
h = Hist(axis0, axis1, axis2, axis3, axis4, axis5)

In [ ]:
# add the axes using the shortcut method
h = (
    Hist.new.Reg(10, -5, 5, overflow=False, underflow=False, name="A")
    .Bool(name="B")
    .Var(range(10), name="C")
    .Int(-5, 5, overflow=False, underflow=False, name="D")
    .IntCat(range(10), name="E")
    .StrCat(["T", "F"], name="F")
    .Double()
)

In [ ]:
assert h.axes[0].name == axis0.name
assert h.axes[1].label == axis1.name  # label will be returned as name if not provided
assert all(h.axes[2].widths == axis2.widths)
assert all(h.axes[3].edges == axis3.edges)
assert h.axes[4].metadata == axis4.metadata
assert all(h.axes[5].centers == axis5.centers)